In [34]:
from textblob import TextBlob
import tweepy
import twitter_credentials as tc
import sys
import re  


#auth
auth = tweepy.OAuthHandler(tc.API_KEY, tc.API_KEY_SECRET)
auth.set_access_token(tc.ACCESS_TOKEN, tc.ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [7]:
search_term = 'world cup 2022'
tweet_amount = 20

tweets = tweepy.Cursor(method=api.search_tweets, q=search_term, lang='en').items(tweet_amount)

In [9]:
polarity = 0

positive = 0
negative = 0
neutral = 0

tweets_clean = []
for tweet in tweets:
    tweet_text = tweet.text.replace('RT', '')       #Remove the RT Tags
    if tweet_text.startswith(' @'):                  #Remove usernames
        position = tweet_text.index(':')
        tweet_text = tweet_text[position+2:]
        tweets_clean.append(tweet_text)
    if tweet_text.startswith('@'):                  #Remove usernames
        position = tweet_text.index(' ')
        tweet_text = tweet_text[position+2:]
        tweets_clean.append(tweet_text)
    #print(tweet_text)
    #Analysis
    analysis = TextBlob(tweet_text)
    tweet_polarity = analysis.polarity
    polarity += tweet_polarity
    if tweet_polarity>0:
        positive +=1
    elif tweet_polarity<0:
        negative+=1
    else:
        neutral+=1

print(polarity)                            # We obtained A polarity of +54 which means the overal sentiment is very positive over the 200 tweets
print("positive tweets : ",positive)
print("negative tweets : ",negative)
print("neutral tweets : ",neutral)


3.5763636363636366
positive tweets :  11
negative tweets :  2
neutral tweets :  7


In [10]:
print(tweets_clean)

['Qatar hosted FIFA World Cup 2022 and gifted #KashmirPashmina shawls to VIPs.\nBrand Kashmir is soaring high as these shawls…', 'Emmy_Wise10 @_AsiwajuLerry Bro just delete this, Which Most world cup assist? Most complete player… https://t.co/AP1DMmpx5e', 'jungkook graduated from uni with high honors, released 4 solo songs and an anthology album with bts, got invited to the whi…', 'Katie McCabe leading Ireland to the 2023 World Cup was truly one of my highlights of 2022 https://t.co/kWWSTWSWWe', 'Soccer legend Pelé died on Thursday at the age of 82.\n\nFor over 60 years Pelé stood alone as Brazil’s all-time leading s…', 'Only 3 players made 20+ tackles at the 2022 World Cup, 2 of those were Enzo Fernandez and Mateo Kovacic (@Squawka) https://t.co…', 'Lionel Messi and his 2022 World Cup. Incredible. https://t.co/YQC9lJ3y0J', 'Two disasters, one near disaster and a World Cup with a body count of thousands. Meanwhile, another faultless year for…', '🕯️ Remembering Pelé 🕯️\n\nTributes flood

In [12]:
all_sentences = []
for word in tweets_clean:
    all_sentences.append(word)
    
all_sentences

lines = list()
for line in all_sentences:    
    words = line.split()
    for w in words: 
       lines.append(w)


print(lines)


['Qatar', 'hosted', 'FIFA', 'World', 'Cup', '2022', 'and', 'gifted', '#KashmirPashmina', 'shawls', 'to', 'VIPs.', 'Brand', 'Kashmir', 'is', 'soaring', 'high', 'as', 'these', 'shawls…', 'Emmy_Wise10', '@_AsiwajuLerry', 'Bro', 'just', 'delete', 'this,', 'Which', 'Most', 'world', 'cup', 'assist?', 'Most', 'complete', 'player…', 'https://t.co/AP1DMmpx5e', 'jungkook', 'graduated', 'from', 'uni', 'with', 'high', 'honors,', 'released', '4', 'solo', 'songs', 'and', 'an', 'anthology', 'album', 'with', 'bts,', 'got', 'invited', 'to', 'the', 'whi…', 'Katie', 'McCabe', 'leading', 'Ireland', 'to', 'the', '2023', 'World', 'Cup', 'was', 'truly', 'one', 'of', 'my', 'highlights', 'of', '2022', 'https://t.co/kWWSTWSWWe', 'Soccer', 'legend', 'Pelé', 'died', 'on', 'Thursday', 'at', 'the', 'age', 'of', '82.', 'For', 'over', '60', 'years', 'Pelé', 'stood', 'alone', 'as', 'Brazil’s', 'all-time', 'leading', 's…', 'Only', '3', 'players', 'made', '20+', 'tackles', 'at', 'the', '2022', 'World', 'Cup,', '2', 'of'

In [19]:
lines = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in lines]           #remove punctuations

lines2 = []

for word in lines:
    if word != '':
        lines2.append(word)

print(lines2)

['Qatar', 'hosted', 'FIFA', 'World', 'Cup', '2022', 'and', 'gifted', 'KashmirPashmina', 'shawls', 'to', 'VIPs', 'Brand', 'Kashmir', 'is', 'soaring', 'high', 'as', 'these', 'shawls', 'EmmyWise10', 'AsiwajuLerry', 'Bro', 'just', 'delete', 'this', 'Which', 'Most', 'world', 'cup', 'assist', 'Most', 'complete', 'player', 'httpstcoAP1DMmpx5e', 'jungkook', 'graduated', 'from', 'uni', 'with', 'high', 'honors', 'released', '4', 'solo', 'songs', 'and', 'an', 'anthology', 'album', 'with', 'bts', 'got', 'invited', 'to', 'the', 'whi', 'Katie', 'McCabe', 'leading', 'Ireland', 'to', 'the', '2023', 'World', 'Cup', 'was', 'truly', 'one', 'of', 'my', 'highlights', 'of', '2022', 'httpstcokWWSTWSWWe', 'Soccer', 'legend', 'Pel', 'died', 'on', 'Thursday', 'at', 'the', 'age', 'of', '82', 'For', 'over', '60', 'years', 'Pel', 'stood', 'alone', 'as', 'Brazils', 'alltime', 'leading', 's', 'Only', '3', 'players', 'made', '20', 'tackles', 'at', 'the', '2022', 'World', 'Cup', '2', 'of', 'those', 'were', 'Enzo', 'Fe

In [56]:
#Stemming

stem = []

for word in lines2:
    wordtostemlist = TextBlob(word).words
    for word2 in wordtostemlist:
        stem.append(word2.stem())
        
stem

['qatar',
 'host',
 'fifa',
 'world',
 'cup',
 '2022',
 'and',
 'gift',
 'kashmirpashmina',
 'shawl',
 'to',
 'vip',
 'brand',
 'kashmir',
 'is',
 'soar',
 'high',
 'as',
 'these',
 'shawl',
 'emmywise10',
 'asiwajulerri',
 'bro',
 'just',
 'delet',
 'thi',
 'which',
 'most',
 'world',
 'cup',
 'assist',
 'most',
 'complet',
 'player',
 'httpstcoap1dmmpx5',
 'jungkook',
 'graduat',
 'from',
 'uni',
 'with',
 'high',
 'honor',
 'releas',
 '4',
 'solo',
 'song',
 'and',
 'an',
 'antholog',
 'album',
 'with',
 'bt',
 'got',
 'invit',
 'to',
 'the',
 'whi',
 'kati',
 'mccabe',
 'lead',
 'ireland',
 'to',
 'the',
 '2023',
 'world',
 'cup',
 'wa',
 'truli',
 'one',
 'of',
 'my',
 'highlight',
 'of',
 '2022',
 'httpstcokwwstwsww',
 'soccer',
 'legend',
 'pel',
 'die',
 'on',
 'thursday',
 'at',
 'the',
 'age',
 'of',
 '82',
 'for',
 'over',
 '60',
 'year',
 'pel',
 'stood',
 'alon',
 'as',
 'brazil',
 'alltim',
 'lead',
 's',
 'onli',
 '3',
 'player',
 'made',
 '20',
 'tackl',
 'at',
 'the',


In [66]:
#Remove StopWords
from nltk.corpus import stopwords

stem2 = []
stop_words = set(stopwords.words('english'))

for word in stem:
    if word not in stop_words:
        stem2.append(word)

stem2

['qatar',
 'host',
 'fifa',
 'world',
 'cup',
 '2022',
 'gift',
 'kashmirpashmina',
 'shawl',
 'vip',
 'brand',
 'kashmir',
 'soar',
 'high',
 'shawl',
 'emmywise10',
 'asiwajulerri',
 'bro',
 'delet',
 'thi',
 'world',
 'cup',
 'assist',
 'complet',
 'player',
 'httpstcoap1dmmpx5',
 'jungkook',
 'graduat',
 'uni',
 'high',
 'honor',
 'releas',
 '4',
 'solo',
 'song',
 'antholog',
 'album',
 'bt',
 'got',
 'invit',
 'whi',
 'kati',
 'mccabe',
 'lead',
 'ireland',
 '2023',
 'world',
 'cup',
 'wa',
 'truli',
 'one',
 'highlight',
 '2022',
 'httpstcokwwstwsww',
 'soccer',
 'legend',
 'pel',
 'die',
 'thursday',
 'age',
 '82',
 '60',
 'year',
 'pel',
 'stood',
 'alon',
 'brazil',
 'alltim',
 'lead',
 'onli',
 '3',
 'player',
 'made',
 '20',
 'tackl',
 '2022',
 'world',
 'cup',
 '2',
 'enzo',
 'fernandez',
 'mateo',
 'kovac',
 'squawka',
 'httpstco',
 'lionel',
 'messi',
 'hi',
 '2022',
 'world',
 'cup',
 'incred',
 'httpstcoyqc9lj3y0j',
 'two',
 'disast',
 'one',
 'near',
 'disast',
 'worl